In [3]:
using Revise

In [4]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using LCPSim
using AtlasRobot
using Blink
using RigidBodyDynamics
using Rotations
using LearningMPC
using LearningMPC.Models
using Gurobi
using StaticArrays
using RigidBodySim

In [5]:
atlas_sim = LearningMPC.Models.PlanarAtlas(:simulation);

In [19]:
mvis = MechanismVisualizer(atlas_sim)
open(mvis, Window())

AtlasRobot.packagepath() = "/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/AtlasRobot/deps"


Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 5556`, ProcessRunning), TCPSocket(RawFD(65) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(2, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(70) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 2, Nullable{Any}(true))))

In [8]:
effort_limiter = let effort_bounds = LCPSim.all_effort_bounds(mechanism(atlas_sim))
    function (τ::AbstractVector, t::Number, state::MechanismState)
        τ .= clamp.(τ, effort_bounds)
        τ
    end
end

bounds_enforcer = let position_bounds = LCPSim.all_configuration_bounds(mechanism(atlas_sim))
    function (τ::AbstractVector, t::Number, state::MechanismState)
        # TODO: handle q̇ vs v correctly
        for i in 1:num_positions(state)
            kp = 1000
            kd = 0.1 * kp
            if configuration(state)[i] > position_bounds[i].upper
                violation = configuration(state)[i] - position_bounds[i].upper
                τ[i] -= kp * violation
                τ[i] -= kd * velocity(state)[i]
            elseif configuration(state)[i] < position_bounds[i].lower
                violation = position_bounds[i].lower - configuration(state)[i]
                τ[i] += kp * violation 
                τ[i] -= kd * velocity(state)[i]
            end
        end
        τ
    end
end

function compose(controllers...)
    function (τ, t, state)
        for c in controllers
            c(τ, t, state)
        end
        τ
    end
end
   
zero_controller(τ::AbstractVector, t::Number, state::MechanismState) = τ .= 0

zero_controller (generic function with 1 method)

In [14]:
atlas_control = LearningMPC.Models.PlanarAtlas(:control);

In [58]:
xstar = nominal_state(atlas_control)
Δt = 0.05
nq = num_positions(xstar)
nv = num_velocities(xstar)
qq = fill(1.0, nq)
qq[1] = 1
qq[2] = 100
qq[3] = 100
qv = fill(0.001, nv)
Q = diagm(vcat(qq, qv))
R = diagm(fill(1e-4, nv))
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, Δt, 
    [Point3D(default_frame(body), 0., 0, 0) for body in 
            findbody.(mechanism(atlas_control), ["r_foot_sole", "l_foot_sole"])])

lqr_inplace = let lqr = lqrsol
    function (τ, t, state)
        τ .= lqr(state)
#         τ .= lqr.u0
    end
end


(::#63) (generic function with 1 method)

In [63]:
mpc_params = LearningMPC.MPCParams(
    Δt=Δt,
    horizon=1,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, TimeLimit=10, MIPGap=1e-2, FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0)
)

lqr_mpc_controller = let scratch = MechanismState(mechanism(atlas_control))
    MPCController(scratch, environment(atlas_control), mpc_params, lqrsol, Function[lqrsol], (s, r) -> nothing)
end

lqr_mpc_inplace = let c = lqr_mpc_controller
    function (τ, t, state)
        τ .= c(state)
    end
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::#69) (generic function with 1 method)

In [64]:
state = nominal_state(atlas_sim)

final_time = 1
Δt = 0.01

composed = PeriodicController(similar(velocity(state)), Δt, compose(lqr_mpc_inplace, effort_limiter))
controlcallback = DiffEqCallbacks.PeriodicCallback(composed)
composed = compose(composed, bounds_enforcer)

dynamics = Dynamics(mechanism(atlas_sim), composed)
problem = ODEProblem(dynamics, state, (0., final_time), callback = CallbackSet(controlcallback, CallbackSet(mvis, state, max_fps = 30.)))

@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = Δt);

884.064491 seconds (215.43 M allocations: 13.202 GiB, 1.35% gc time)


In [67]:
RigidBodySim.animate(mvis, state, sol)

### Next Steps

At least part of the problem when running on the full Atlas is that the nominal input is almost always wrong by enough to disrupt the sytsem, and turning R down so that the nominal is ignored results in crazy accelerations. Instead, we should essentially feedback linearize the system by applying the LQR "input" cost to $\dot{v}$ instead of u (with a nominal value of 0 always)

In [ ]:
lqr_mpc_controller = let scratch = MechanismState(atlas)
    simple_env = LCPSim.Environment{Float64}(
    [
        (body, Point3D(default_frame(body), offset...), floor) for body in feet for offset in [[0., 0, 0]]
    ]
);
    MPCController(scratch, simple_env, mpc_params, lqrsol, Function[lqrsol], (s, r) -> nothing)
end

In [38]:
x0 = init_atlas!(MechanismState(atlas))
lqrsol(x0)

configuration(state) = [0.0, 0.85, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.45, -0.45, 0.0, 0.0, 0.0, 1.1, 1.1, 0.0, 0.0, -0.65, -0.65, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


33-element Array{Float64,1}:
   0.0        
   0.0        
   0.0        
   0.0        
   0.0        
   0.0        
  -7.5305     
 -17.2696     
  17.2696     
   1.90522    
 -86.514      
 -86.514      
   0.0        
   ⋮          
  -0.00381321 
  -0.00564618 
  -1.46022    
   1.46022    
   9.93221    
  -9.93221    
  -0.00381321 
  -0.00564618 
   0.749362   
  -0.749362   
   0.000916489
  -0.000916489

In [39]:
lqrsol.u0

33-element Array{Float64,1}:
   0.0        
   0.0        
   0.0        
   0.0        
   0.0        
   0.0        
  -7.5305     
 -17.2696     
  17.2696     
   1.90522    
 -86.514      
 -86.514      
   0.0        
   ⋮          
  -0.00381321 
  -0.00564618 
  -1.46022    
   1.46022    
   9.93221    
  -9.93221    
  -0.00381321 
  -0.00564618 
   0.749362   
  -0.749362   
   0.000916489
  -0.000916489

In [ ]:
x0 = init_atlas!(MechanismState(atlas))
# set_velocity!(x0, findjoint(atlas, "pelvis_to_world"), [-0.5, 0, 0])
results = LCPSim.simulate(x0, x -> begin
        set_configuration!(mvis, configuration(x))
        lqrsol.u0
    end, env, 0.01, 100, mpc_params.lcp_solver);
@show length(results)
setanimation!(mvis, results)


In [31]:
results[end].state.configuration

33-element SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}:
 -0.0033903  
  0.928234   
 -0.00107767 
 -0.00444538 
 -0.000581757
  0.000814242
 -0.0055727  
  0.00503378 
  0.0052237  
 -0.00961241 
  0.109388   
  0.109312   
  0.00720999 
  ⋮          
  0.000270581
  0.0141765  
 -0.00395611 
 -0.00414603 
  0.107546   
 -0.180345   
  0.00337585 
 -0.0113241  
 -0.0627932  
  0.0474613  
 -0.000682655
  0.000796706

In [33]:
lqrsol.x0[1:num_positions(atlas)]

33-element Array{Float64,1}:
  0.0 
  0.85
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
 -0.45
 -0.45
  0.0 
  ⋮   
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 
  0.0 

In [36]:
for i in 1:num_positions(atlas)
    println(results[end].state.configuration[i], " ", lqrsol.x0[i], " ", LCPSim.all_configuration_bounds(atlas)[i])
end

-0.0033903022595943777 0.0 (-10.0, 10.0)
0.9282342165807665 0.85 (-10.0, 10.0)
-0.0010776723324969694 0.0 (-10.0, 10.0)
-0.004445382594485738 0.0 (-0.663225, 0.663225)
-0.0005817569492283332 0.0 (-0.174358, 0.786794)
0.0008142423379124708 0.0 (-0.786794, 0.174358)
-0.005572699054055387 0.0 (-0.219388, 0.538783)
0.005033782171372851 0.0 (-0.523599, 0.523599)
0.005223701629129093 0.0 (-0.523599, 0.523599)
-0.00961240706024892 0.0 (-0.523599, 0.523599)
0.10938777178672855 -0.45000000000000007 (-1.61234, 0.65764)
0.10931225296645976 -0.45000000000000007 (-1.61234, 0.65764)
0.007209986997368364 0.0 (-1.5708, 0.785398)
-0.17322956377728577 0.0 (-0.602139, 1.14319)
0.011322163704369385 0.0 (-0.785398, 1.5708)
0.0 1.1 (0.0, 2.35637)
0.0 1.1 (0.0, 2.35637)
-0.1578296389010598 0.0 (-1.5708, 1.5708)
0.13970895507375755 0.0 (-1.5708, 1.5708)
-0.10938714483862708 -0.65 (-1.0, 0.7)
-0.1093131304601527 -0.65 (-1.0, 0.7)
0.0002705812247298092 0.0 (0.0, 3.14159)
0.014176527924943986 0.0 (0.0, 3.14159)


In [108]:
for r in results[end-1].joint_contacts
    println(r)
end

LCPSim.JointLimitResult{Float64,Float64}([-2.03727e-10, -0.0, -2.76486e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.33172e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.33513e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.33513e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.33172e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.21007e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.21007e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.54659e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.33399e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.33399e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.32873e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.32832e-10], 1717.9072268399998)
LCPSim.JointLimitResult{Float64,Float64}([-2.32873e-10],

In [50]:
ts, qs, vs = RigidBodyDynamics.simulate(init_atlas!(MechanismState(atlas)), 1.0, 
    ((τ, t, state) -> τ .= 0); 
    Δt=0.001)

([0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009  …  0.991, 0.992, 0.993, 0.994, 0.995, 0.996, 0.997, 0.998, 0.999, 1.0], RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}[[0.0, 0.85, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1.29942e-22, 0.849995, 2.25597e-21, 2.23466e-22, 8.09443e-22, -8.91671e-23, 2.63518e-22, -1.33081e-21, -2.83498e-21, -4.33543e-21  …  -7.53141e-21, 7.69418e-21, 3.76952e-20, -6.51101e-20, -6.9808e-20, 1.82795e-19, -5.56863e-19, 5.75447e-19, -1.59134e-19, -7.41255e-20], [-5.18957e-22, 0.84998, 9.10823e-21, 8.79209e-22, 2.22849e-21, 6.48965e-22, 1.05984e-21, -5.27853e-21, -1.15549e-20, -1.74817e-20  …  -3.22727e-20, 3.2918e-20, 1.5143e-19, -2.60436e-19, -2.7112e-20, 7.31179e-19, -2.23202e-18, 2.30179e-18, -8.16799e-19, -2.96502e-19], [-1.16542e-21, 0.849956, 2.04606e-20, 1.95565e-21, 2.82374e-21, 3.6447

In [51]:
setanimation!(mvis, ts, qs)

true